In [18]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import cv2
from sklearn.model_selection import train_test_split

from PIL import Image
import PIL
import torchvision

import os

In [4]:
def get_file_name(base):
    for path, dir, file in os.walk(base):
        for f in file:
            if ".png" in f:
                yield(os.path.join(path, f))

In [45]:
fake_type = []
for idx, file_path in enumerate(get_file_name('./data/manipulated/')):
    fake_type.append(((file_path.split('/'))[-1]).split('_')[0])
    
fake_type = np.array(fake_type)
fake_type = np.unique(fake_type).tolist()
print(fake_type)
# fake_type = np.array(np.unique(fake_type, return_counts=True)).T
# for i, t in enumerate(fake_type):
#     print(i+1, t)
# fake_type = ((fake_type.T)[0]).tolist()

(['DF', 'F2F', 'FS', 'NT', 'eyes', 'mouth'], list)

In [40]:
GENERAL_TRANSFORM = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.AutoAugment(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.A
    torchvision.transforms.Normalize([0.5]*3, [0.5]*3)
])


img_data = [] # 12000 * (3, 299, 299)
img_label = [] # 1D array

# 0 real
# 1 DF
# 2 F2F
# 3 FS
# 4 NT
# 5 eyes
# 6 mouth

for idx, file_path in enumerate(get_file_name('./data/')):
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img)
    
    img = GENERAL_TRANSFORM(img)
    img_data.append(img)
    
    if 'manipulated' in file_path: 
        for i, t in enumerate(fake_type):
            if t in file_path:
                img_label.append(i+1)
                break
    else:
        img_label.append(0)
        

img_label = np.asarray(img_label)
np.array(np.unique(img_label, return_counts=True)).T

array([[   0, 4000],
       [   1, 1333],
       [   2, 1335],
       [   3, 1333],
       [   4, 1333],
       [   5, 1333],
       [   6, 1333]])

In [41]:
X_train, X_test, y_train, y_test = train_test_split(img_data, img_label, test_size = 0.2, random_state = 4487)

train_batchsize = 32
test_batchsize = 1

X_train, y_train = torch.stack(X_train), torch.Tensor(y_train)
X_test, y_test = torch.stack(X_test), torch.Tensor(y_test)
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = train_batchsize, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = test_batchsize, shuffle = True)

In [49]:
TRAIN_TRANSFORM = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(),
    # torchvision.transforms.AutoAugment(),
])

for img in X_train:
    img = TRAIN_TRANSFORM(img)

In [48]:
t = [.1, .2, .3, .456789]
t = pd.DataFrame(t); t.to_csv("t.csv", index=False, header=False)
t

,0
0,0.100000
1,0.200000
2,0.300000
3,0.456789


In [50]:
fake_type = []
for idx, file_path in enumerate(get_file_name("./data/manipulated/")):
    fake_type.append(((file_path.split("/"))[-1]).split("_")[0])
    
fake_type = np.array(fake_type)
fake_type = np.unique(fake_type).tolist()
print(0, "real")
for i, t in enumerate(fake_type):
    print(i+1, t)

0 real
1 DF
2 F2F
3 FS
4 NT
5 eyes
6 mouth


In [53]:
GENERAL_TRANSFORM = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
])

TRAIN_TRANSFORM = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(),
    # torchvision.transforms.AutoAugment(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5]*3, [0.5]*3)
])
TEST_TRANSFORM = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5]*3, [0.5]*3)
])

img_data = [] # 12000 * (3, 299, 299)
img_label = [] # 1D array

# 0 real
# 1 DF
# 2 F2F
# 3 FS
# 4 NT
# 5 eyes
# 6 mouth

for idx, file_path in enumerate(get_file_name("./data/")):
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img)
        
    img = GENERAL_TRANSFORM(img)
    img_data.append(img)
    
    if "manipulated" in file_path: 
        for i, t in enumerate(fake_type):
            if t in file_path:
                img_label.append(i+1)
                break
        
    else:
        img_label.append(0)
        

img_label = np.asarray(img_label)
print(np.array(np.unique(img_label, return_counts=True)).T)


[[   0 4000]
 [   1 1333]
 [   2 1335]
 [   3 1333]
 [   4 1333]
 [   5 1333]
 [   6 1333]]


In [62]:
# train_test_split
# TODO: is the split balance?
X_train, X_test, y_train, y_test = train_test_split(img_data, img_label, test_size = 0.2, random_state = 4487)
for i,_ in enumerate(X_train):
    X_train[i] = TRAIN_TRANSFORM(X_train[i])
for i,_ in enumerate(X_test):
    X_test[i] = TRAIN_TRANSFORM(X_test[i])

In [64]:
torch.save({
    "X_train": X_train,
    "X_test": X_test,
    "y_train": y_train,
    "y_test": y_test
}, "./resnet50_2_split_data.pkl")

In [65]:
checkpoint = torch.load("./resnet50_2_split_data.pkl")

In [66]:
checkpoint["X_train"]

[tensor([[[ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          ...,
          [-0.7333, -0.7255, -0.7098,  ..., -0.5294, -0.6235, -0.6549],
          [-0.7255, -0.7176, -0.7098,  ..., -0.5373, -0.5922, -0.6157],
          [-0.7255, -0.7098, -0.7020,  ..., -0.5216, -0.5686, -0.5922]],
 
         [[ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          ...,
          [-0.6784, -0.6784, -0.6706,  ..., -0.6392, -0.7176, -0.7490],
          [-0.6863, -0.6784, -0.6706,  ..., -0.6549, -0.7020, -0.7176],
          [-0.7020, -0.6863, -0.6784,  ..., -0.6471, -0.6784, -0.6941]],
 
         [[ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  1.0000,

In [67]:
class Meso4(nn.Module):
    def __init__(self):
        super(Meso4, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 8, 3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(8)
        self.maxpooling1 = nn.MaxPool2d(kernel_size=(2, 2))
        
        self.conv2 = nn.Conv2d(8, 8, 5, padding=2, bias=False)
        
        self.conv3 = nn.Conv2d(8, 16, 5, padding=2, bias=False)
        self.bn3 = nn.BatchNorm2d(16)
        
        self.conv4 = nn.Conv2d(16, 16, 5, padding=2, bias=False)
        self.maxpooling4 = nn.MaxPool2d(kernel_size=(4, 4))
        
        self.dropout = nn.Dropout2d(0.5) # dropout or dropout2d?
        self.fc1 = nn.Linear(16*8*8, 16)
        self.fc2 = nn.Linear(16, 2)
        
        self.relu = nn.ReLU() # is inplace needed?
        self.leakyrelu = nn.LeakyReLU(0.1)
        
    def forward(self, input):
        # input should be (3, 256, 256)
        
        x = self.conv1(input)
        x = self.relu(x)
        x = self.bn1(x)
        x = self.maxpooling1(x)
        
        x = self.conv2(x)
        x = self.relu(x)
        x = self.bn1(x)
        x = self.maxpooling1(x)
        
        x = self.conv3(x)
        x = self.relu(x)
        x = self.bn3(x)
        x = self.maxpooling1(x)
        
        x = self.conv4(x)
        x = self.relu(x)
        x = self.bn3(x)
        x = self.maxpooling4(x)
        
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.leakyrelu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x
        

In [68]:
model = Meso4()

In [ ]:
model.cuda()

In [70]:
t = torch.Tensor([.1, .2, .15, .15, .4])
print(t)
t = torch.Tensor([t[0], 1-t[0]])

print(t)

tensor([0.1000, 0.2000, 0.1500, 0.1500, 0.4000])
tensor([0.1000, 0.9000])


In [87]:
t = torch.Tensor([.1, .2, .15, .15, .4, 0])
# t = torch.stack([t, t, t, t, t])
print(t)

for i in range(t.shape[0]):
    t[i] = 0 if t[i] == .1 else 1
t

tensor([0.1000, 0.2000, 0.1500, 0.1500, 0.4000, 0.0000])


tensor([0., 1., 1., 1., 1., 1.])

In [88]:
t[0]

tensor(0.)